## 2-1 로그인이 필요한 사이트에서 다운받기
### 이번 절에서 배울 내용
* 로그인의 구조
* requests 모듈

### HTTP 통신
* 웹브라우저와 웹서버는 HTTP라고 불리는 통신 규약을 사용
* HTTP 통신은 브라우저에서 서버로 요청(request) 하고, 서버에서 브라우저로 응답(response) 할 때 어떻게 할 것인지를 나타내는 규약
* 예를 들어 웹 브라우저로 <http://www.naver.com> 이라는 URL에 접근한다고 가정하자. 먼저 웹 브라우저는 www.naver.com 이라는 웹서버를 찾음. 웹서버가 발견되면 index.html 이라는 파일을 보고 싶다고 요청을 보냄. naver.com 이라는 서버가 이러한 요청을 받으면 index.html 파일의 내용을 응답해줌. 
* 요청에 대해 응답을 돌려주는 굉장히 간단한 구조로 기본적으로 무상태(stateless) 통신이다. 무상태 통신이란 같은 URL에 여러번 접근해도 같은 데이터를 돌려주는 통신을 나타낸다. 조금 더 자세히 말하면 이전에 어떤 데이터를 가져갔는 지 등에 대한 정보(상태:state)를 전혀 저장하지 않는 통신임

### 쿠키
* 무상태 HTTP 통신으로는 회원제 사이트를 만들 수 없음. 과거의 정보가 저장되지 않으면 장바구니에 어떤 상품을 추가했는지 등도 구현할 수 없음.
* 그래서 웹 브라우저에 쿠키(Cookie)라는 구조가 추가됨. 이는 웹 브라우저를 통해 사이트에 방문하는 사람의 컴퓨터에 일시적으로 데이터를 저장하는 기능임. 다만 쿠키에는 제약이 있음. 데이터를 마음대로 저장할 수 있는 것이 아니라 1개의 쿠키에 저장할 수 있는 데이터의 크기가 4096바이트로 제한됨. 그리고 쿠키는 HTTP 통신 헤더를 통해 읽고 쓸 수 있음. 따라서 방문자 또는 확인자 측에서 원하는 대로 변경할 수 있음. 

<img src="cookie.png" width=400 height=600>

* 이처럼 쿠키는 HTTP 헤더를 기반으로 이뤄지므로 방문자가 데이터를 원하는 대로 변경할 수 있다는 성질이 있음. 따라서 변경하면 문제가 생길 수 있는 비밀번호 등의 비밀 정보를 저장하기에는 알맞지 않음.
* 따라서 세션이라는 구조를 사용하게 됨
* 세션도 쿠키를 사용해 데이터를 저장한다는 점은 같음. 하지만 쿠키에는 방문자 고유 ID만 저장하고, 실제로 모든 데이터는 웹 서버에 저장한다. 서버에 데이터를 저장하므로 쿠키와는 다르게 저장할 수 있는 데이터에 제한이 없다는 것이 중요한 특징임.
* HTTP 통신은 무상태 통신이지만 세션을 이용하면 쿠키에 기록되어 있는 고유 ID를 키로 사용해 상태를 변수를 확인할 수 있음. 따라서 통신을 계속해서 진행하는 것 같은 상태 유지(stateful) 통신을 구현할 수 있음. 이러한 세션이라는 구조를 이용하면 회원제 웹 사이트 또는 쇼핑몰 사이트를 구현할 수 있음.
* 세션의 구성은 다음과 같음. 일단 방문자가 처음 웹사이트에 방문할 때, 쿠키에 세션 ID를 기록함. 그리고 웹 서버에는 세션 ID를 기반으로 데이터 파일을 만들고, 여기에 저장하고 싶은 변수의 값을 저장해 둠. 그리고 사용자가 다시 서버에 접근하면 쿠키의 세션 ID를 기반으로 저장한 변수를 확인하는 것임.
<img src="cookie3.png" width=400 height=600>

### requests를 사용해보기
* urllib.request로도 쿠키를 이용한 접근이 가능하지만 방법이 복잡하여 requests라는 패키지를 사용하여 쿠키를 이용한 접근을 한다.
* 쉽게 로그인할 수 없게 보안적으로 구성된 네이버나 나듬 등의 퐅털 사이트들은 이 방법으로 로그인할 수 없음.
* pip install requests
* 한빛출판네트워크 사이트를 이용해보자.
    - 로그인 페이지: <http://www.hanbit.co.kr/member/login.html>
    - 마이페이지: <http://www.hanbit.co.kr/myhanbit/myhanbit.html>
* 아이디와 비밀번호 입력하는 부분에 마우스 오른쪽 버튼을 클릭하고 검사를 누르면 다음과 같은 입력양식을 확인할 수 있음.

```html
<form name="frm"  id="frm"  action="#" method="post">
		<input name="retun_url" id="retun_url" type="hidden" value="" class="i_text" size="100" >
		<div class="login_left">		
			<fieldset>
				<legend>한빛출판네트워크 로그인</legend>
				
				<label class="i_label" for="login_id"><strong></strong>
					<input name="m_id" id="m_id" type="text" value="" class="i_text" placeholder="아이디" onkeydown="javascript:if(event.keyCode==13){login_proc(); return false;}">
				</label> 

				<label class="i_label" for="login_pw"><strong></strong>
					<input name="m_passwd"  id="m_passwd" type="password" value="" class="i_text" placeholder="비밀번호" onkeydown="javascript:if(event.keyCode==13){login_proc(); return false;}">
				</label>
				
				<label>
					<input  type="button" name="login_btn"  id="login_btn" value="로그인" class="btn_login" >					
				</label>
				
				<label class="i_label2">
					<input type="checkbox" name="keepid" id="keepid" value="1" class="i_check"><strong>아이디 저장</strong>
				</label>
			</fieldset>
			
			<ul class="login_btn">
				<li><a href="/member/find_id.html" class="btn_idc">아이디 찾기</a></li>
				<li><a href="/member/find_pw.html" class="btn_pwc">비밀번호 찾기</a></li>
				<li><a href="/member/member_agree.html" class="btn_joinc">회원가입</a></li>
			</ul>
		</div>
		</form>

```

* 입력 양식으로 m_id, m_passwd라는 값(name 속성의 값)을 입력하고, 입력 양식을 제출하면 로그인되는 구조를 알 수 있음.
* 로그인하고 마이페이지에 들어가서 마일리지 (.mileage_section1 > span) 와 한빛이코인(.mileage_section2 > span)을 출력해보자.

In [1]:
# 로그인을 위한 모듈 추출하기
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# 아이디와 비밀번호 지정하기[자신의 것을 사용해주세요] --- (※1)
USER = "<아이디>"
PASS = "<비밀번호>"
# 세션 시작하기 --- (※2)
session = requests.session()
# 로그인하기 --- (※3)
login_info = {
    "m_id": USER,  # 아이디 지정
    "m_passwd": PASS  # 비밀번호 지정
}
url_login = "http://www.hanbit.co.kr/member/login_proc.php"
res = session.post(url_login, data=login_info)
res.raise_for_status() # 오류가 발생하면 예외가 발생합니다.
# 마이페이지에 접근하기 --- (※4)
url_mypage = "http://www.hanbit.co.kr/myhanbit/myhanbit.html" 
res = session.get(url_mypage)
res.raise_for_status()
# 마일리지와 이코인 가져오기 --- (※5)
soup = BeautifulSoup(res.text, "html.parser")
mileage = soup.select_one(".mileage_section1 span").get_text()
ecoin = soup.select_one(".mileage_section2 span").get_text()
print("마일리지: " + mileage)
print("이코인: " + ecoin)

마일리지: 2,000
이코인: 0


1. 아이디와 비밀번호 지정한다. 자신의 것으로 입력한다.
1. session() 메서드를 사용해 사용자 세션 시작. 
1. 로그인한다. 로그인을 위해 이전에 분석한 입력 양식의 name 속성과 값을 입력한다. 그리고 로그인 전용 URL에 POST 요청을 수행한다.
1. 로그인이 완료되면 마이페이지에 접근한다. 
1. 마이페이지의 내용을 출력한다.
1. request.session을 이용해 쿠키를 사용하는 회원제 사이트에 로그인하는 것이 가능함.

### requests 모듈의 메서드
* import requests 로 모듈을 읽어들임
* HTTP에서 사용하는 GET, POSt 등의 메서드는 requests 모듈에 같은 이름의 메서드가 있음.
* 그 밖에도 PUT/DELETE/HEAD 등의 요청을 위한 메서드도 있음.


In [4]:
# GET 요청
r=requests.get("http://google.com")

# POST 요청
formdata={"key1":"value1", "key2":"value2"}
r=requests.post("http://example.com", data=formdata)
r=requests.put("http://httpbin.org/put")
r=requests.delete("http://httpbin.org/delete")
r=requests.head("http://httpbin.org/get")

* GET, POST 등의 리턴값에 있는 text와 content 속성을 참조하면 내부의 데이터를 확인할 수 있음
* 다음 프로그램은 현재 시간에 대한 데이터를 추출하고, 추출한 데이터를 텍스트 형식과 바이너리 형식으로 출력하는 예제
* 여기서 <http://api.aoikujira.com/time/get.php>는 책에서 제공하는 시간확인 웹 API 이다. 웹 브라우저에 주소를 입력해서 들어가보면 어떤 것을 출력하는지 확인 가능함.

In [5]:
# 데이터 가져오기
import requests
r = requests.get("http://api.aoikujira.com/time/get.php")
# 텍스트 형식으로 데이터 추출하기
text = r.text
print(text)
# 바이너리 형식으로 데이터 추출하기
bin = r.content
print(bin)

2018/09/18 01:41:49
b'2018/09/18 01:41:49'


* 첫번째 출력은 일반 텍스트이고, 두번째 출력은 binary 형태임. 
* 다음은 바이너리를 활용해 바이너리 데이터인 이미지를 받아 저장하는 예제임.

In [7]:
# 이미지 데이터 추출하기
import requests
r = requests.get("http://wikibook.co.kr/wikibook.png")
# 바이너리 형식으로 데이터 저장하기
with open("test4.png", "wb") as f:
    f.write(r.content)
print("saved")

saved
